# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats import weightstats as stests
from scipy.stats import chi2_contingency

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

### 1. What test is appropriate for this problem? Does CLT apply?

In [15]:
print('total observations: ', data.id.count())
black_names = data[data.race=='b']
white_names = data[data.race=='w']
print('black name observations: ', black_names.id.count())
print('white name observations: ', white_names.id.count())
#number of callbacks
black_call_back = len(data[(data.race=='b') & (data.call==1)])
white_call_back = len(data[(data.race=='w') & (data.call==1)])
print('callbacks for black sounding names, ', black_call_back)
print('callbacks for white sounding names, ', white_call_back)

total observations:  4870
black name observations:  2435
white name observations:  2435
callbacks for black sounding names,  157
callbacks for white sounding names,  235


The CLT applies since the total sample size as well as the samples for black and white names are all bigger than 30. 
The distribution is a binomial distribution and the normal conditions applies because the occurrence of successes (157 and 235) and failures (2278 and 2200) are both bigger than 10. 
A z-statistic hypothesis test is appropriate for this problem. 

### 2. What are the null and alternate hypotheses?

H0: there is no difference in the rate of callbacks between black sounding names and white sounding names   
HA: there is significant difference in the rate of callbacks between black sounding names and white sounding names  
Significance level: 0.05

In [5]:
# Your solution to Q3 here

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

##### bootstrapping

In [6]:
# number of callbacks for black-sounding names
b_call = data[data.race=='b'].call
b_call_rate = np.mean(b_call)
# number of callbacks for white-sounding names
w_call = data[data.race=='w'].call
w_call_rate = np.mean(w_call)
print('call back rate for black sounding names: ', b_call_rate)
print('call back rate for white sounding names: ', w_call_rate)

call back rate for black sounding names:  0.0644763857126236
call back rate for white sounding names:  0.09650924056768417


In [7]:
sample_call_rate_diff = w_call_rate - b_call_rate
sample_call_rate_diff

0.03203285485506058

In [8]:
## shift b_call and w_call to have the same mean
call_rate = np.mean(data.call)
b_call_shifted = b_call + (call_rate - b_call_rate)
w_call_shifted = w_call + (call_rate - w_call_rate)

## bootstrap 10000 samples
def draw_bs_reps(data, func, size=1):
    bs_reps = np.empty(size)
    
    for i in range(size):
        bs_samples = np.random.choice(data, len(data))
        bs_reps[i] = func(bs_samples)
    return bs_reps

bs_replicates_b = draw_bs_reps(b_call_shifted, np.mean, 10000)
bs_replicates_w = draw_bs_reps(w_call_shifted, np.mean, 10000)
bs_replicates_diff = bs_replicates_w - bs_replicates_b

p = np.sum(bs_replicates_diff >= sample_call_rate_diff)/len(bs_replicates_diff)
print('p-value: ', p)

## confidence interval 
conf_int = np.percentile(bs_replicates_diff, [2.5, 97.5])
print('confidence interval: ', conf_int)

## margin of error
me = (conf_int[1] - conf_int[0])/2
print('margin of error ', me)

p-value:  0.0
confidence interval:  [-0.01519511  0.01560573]
margin of error  0.015400417149066925


##### frequentist statistical approach

In [9]:
mu = sample_call_rate_diff
se = np.sqrt((np.var(b_call) + np.var(w_call))/len(b_call))
conf_int_s = stats.norm.interval(0.95, loc=mu, scale=se)
me_s = (conf_int_s[1] - conf_int_s[0])/2
tstat, pval = stats.ttest_ind(b_call, w_call)

print('standard error: ', se)
print('margin of error: ', me_s)
print('confidence interval: ', conf_int_s)
print('p-value: ', pval)

standard error:  0.007783308165446165
margin of error:  0.015255003684851005
confidence interval:  (0.016777851170209572, 0.04728785853991158)
p-value:  3.940802103128886e-05


In [10]:
## Chi-square test

x = [[sum(w_call), (len(white_names)-sum(w_call))], 
     [sum(b_call), (len(black_names)-sum(b_call))]]

chi2, pval, dof, expected = chi2_contingency(x)
print('p-value: ', pval)

p-value:  4.997578389963255e-05


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

### 4. Write a story describing the statistical significance in the context or the original problem.

The p-value we got from the hypothesis test is less than the significance level of 0.05. We can thus reject the null hypothesis. We can conclude that there is a significant difference in the callback rates between black-sounding names and white-sounding names. Therefore, the racial discrimination exists.

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

My analysis concludes that there is a significant difference in the callback success between the two groups, but it doesn't suggest that the race/name is the most important factor. There could be other factors in the resume that also contribute to the difference. We need to investigate the other variables to make the final conclusion.